In [1]:
import os
%pwd

'c:\\Users\\49179\\Desktop\\Food_image_classification\\research'

In [2]:
os.chdir("../")

os.getcwd()

'c:\\Users\\49179\\Desktop\\Food_image_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path
import torch


@dataclass
class training_config:
    root_dir: Path
    model_path: Path
    epochs: int
    batch_size: int
    learning_rate: float
    device: str



In [4]:
from Food_Classification.utils.common import read_yaml, create_directory
from Food_Classification.constants import *

class ConfigurationManager:
    def __init__(self, 
                 config_file_path= CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directory([self.config.artifacts_root])


    def get_training_config(self) -> training_config:
        config = self.config.model_training
        create_directory([config.root_dir])
        self.model_path = self.config.prepare_base_model.updated_base_model
        Training_Config = training_config(root_dir= config.root_dir,
                                          model_path= self.model_path,
                                          epochs= self.params.EPOCHS,
                                          batch_size= self.params.BATCH_SIZE,
                                          learning_rate= self.params.LEARNING_RATE,
                                          device=self.params.DEVICE
                                         )


        return Training_Config



In [10]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter
import torch



class Model_training:
    def __init__(self, config: training_config, train_dataloader: DataLoader, test_dataloader: DataLoader, loss_function: torch.nn.Module, writer: SummaryWriter):
        self.config = config
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader
        self.model = torch.load(self.config.model_path,map_location=torch.device(self.config.device))
        self.loss_function = loss_function
        self.optimizer = self.config.optimizer
        self.writer = writer
    
    def train_step(self,train_dataloader):
        model = self.model
        try:
            # Initiate model training
            model.train()
            train_loss = 0
            train_acc= 0
            progress = tqdm(train_dataloader)
            # Looping through batches
            for batch, (data, target) in enumerate(progress):
                # Setting up data into device
                data, target = data.to(self.config.device), target.to(self.config.device)

                # forward propogation
                target_pred = model(data)

                # Calculating loss
                loss = self.loss_function(target_pred, target)
                train_loss += loss.item()
                
                # setting optimizer to zero gradient
                self.optimizer.zero_grad()
                
                # backward propogation
                loss.backward()
                
                # updating weights
                self.optimizer.step()


                
                # Calculating accuracy
                target_pred_class = torch.argmax(torch.softmax(target_pred, dim=1), dim=1) 
                train_acc += (target_pred_class == target).sum().item() / len(target_pred)

            train_loss /= len(train_dataloader)
            train_acc /= len(train_dataloader)

            progress.set_description(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
            return train_loss, train_acc
                
            
            
        except Exception as e:
            raise e


    def test_step(self, test_dataloader):
        model = self.model
        try:
            # Intiatig model evaluation
            model.eval()

            test_loss, test_acc = 0,0
            progress = tqdm(test_dataloader)

            with torch.inference_mode():
                for batch, (data, target) in enumerate(progress):
                    data, target = data.to(self.config.device), target.to(self.config.device)

                    # forward propogation
                    target_pred = model(data)

                    # Calculating Loss
                    loss = self.loss_function(target_pred, target)
                    test_loss += loss.item()

                    # calculating Accuracy
                    target_pred_class = torch.argmax(target_pred, dim =1)
                    test_acc = (target_pred_class == target).sum().item()/len(target_pred)

                test_loss /= len(test_dataloader)
                test_acc /= len(test_dataloader)

                progress.set_description(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
                return test_loss, test_acc
        except Exception as e:
            raise e


    def initiate_training(self):
        model = self.config.model_path
        results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
        }

        for epoch in range(self.config.epochs):
            train_loss, train_acc = self.train_step(self.train_dataloader)
            test_loss, test_acc = self.test_step(self.test_dataloader)

            # Update results dictionary
            results["train_loss"].append(train_loss)
            results["train_acc"].append(train_acc)
            results["test_loss"].append(test_loss)
            results["test_acc"].append(test_acc)


         ### Experiment tracking
        self.writer.add_scalars(main_tag="Loss",
                            tag_scalar_dict = {"Train_loss" : self.train_loss,
                                                "Test_loss" : self.test_loss},
                            global_step= epoch
                            )
        self.writer.add_scalars(main_tag="Accuracy",
                            tag_scalar_dict = {"Train_acc" : self.train_acc,
                                                "Test_acc" : self.test_acc},
                            global_step= epoch
                            )

        self.writer.add_graph(model=model,
        input_to_model = torch.randn(32,3,224,224).to(self.config.device))
    # Return the filled results at the end of the epochs
        return results

In [11]:
from Food_Classification.components.data_transformation import data_transformation
from Food_Classification.components.tensorboard import preparetensorboard
from Food_Classification.components.prepare_base_model import PrepareBaseModel
from Food_Classification.config.configuration import ConfigurationManager
config = ConfigurationManager()
transformation_config = config.get_data_transform_config()
transform = data_transformation(config=transformation_config)
train_dataloader, test_dataloader, class_names = transform.initiate_data_transformation()
tensorboard = preparetensorboard(config=config.get_tensorboard_config())
writer = tensorboard.get_summary_writer()
prepare_base_model_config = config.get_base_model_config()
prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
prepare_base_model.get_base_model()
prepare_base_model.update_base_model()
train_config = config.get_training_config()
training = Model_training(train_config, train_dataloader=train_dataloader, test_dataloader=test_dataloader, loss_function= torch.nn.CrossEntropyLoss(),writer=writer)
training.initiate_training()

[2024-05-06 22:53:15,462: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-05-06 22:53:15,467: INFO: common: yaml file params.yaml loaded successfully]
[2024-05-06 22:53:15,470: INFO: common: directory artifacts created successfully]
[2024-05-06 22:53:15,471: INFO: common: directory artifacts/data_transforms created successfully]
[2024-05-06 22:53:15,472: INFO: data_transformation: Initiating data transformation]
[2024-05-06 22:53:15,473: INFO: data_transformation: Transforming train data]
[2024-05-06 22:53:15,474: INFO: data_transformation: Transforming test data]
[2024-05-06 22:53:15,479: INFO: data_transformation: creating_dataloaders]
[2024-05-06 22:53:15,534: INFO: data_transformation: DataLoaders created]
[2024-05-06 22:53:15,535: INFO: common: directory artifacts/prepare_tensorboard created successfully]
[2024-05-06 22:53:15,536: INFO: common: directory artifacts/prepare_tensorboard/tensorboard_log_dir created successfully]
[2024-05-06 22:53:15,542: INFO: co

  0%|          | 0/188 [01:11<?, ?it/s]


AttributeError: 'str' object has no attribute 'zero_grad'